# **ASSIGNMENT 4**




In [1]:
from google.colab import drive

In [2]:

drive.mount('/content/drive')

Mounted at /content/drive


## 1. Load the dataset

In [3]:
dataset_location="/content/drive/MyDrive/Colab Notebooks/spam.csv"

## 2. Import the library

In [4]:
import pandas as pd
import nltk
import re
import numpy as np

In [5]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.translate.ribes_score import word_rank_alignment

In [6]:
from numpy.lib.shape_base import split
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

In [7]:
from keras.layers import LSTM,Dense,Dropout,Input,Embedding,Activation,Flatten
from keras.models import Model

In [8]:
import nltk

## 3. Read dataset and do preprocessing

In [9]:
data = pd.read_csv(dataset_location,encoding = "ISO-8859-1")

In [10]:
data.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis = 1,inplace = True)

In [11]:
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [12]:
nltk.download('stopwords',quiet=True)

True

In [13]:
nltk.download('all',quiet=True)

True

In [14]:
ps = PorterStemmer()
input = []

In [15]:
for i in range(0,5572):
  v2 = data['v2'][i]

  #removing punctuation
  v2 = re.sub('[^a-zA-Z]',' ',v2)

  #converting to lower case
  v2 = v2.lower()

  #splitting the sentence
  v2 = v2.split()

  #removing the stopwords and stemming
  v2 = [ps.stem(word) for word in v2 if not word in set(stopwords.words('english'))]

  v2 = ' '.join(v2)

  input.append(v2)

## 4.Create Model

In [16]:
#creating document term matrix
cv = CountVectorizer(max_features=2000)
x = cv.fit_transform(input).toarray()
x.shape

(5572, 2000)

In [17]:
l = preprocessing.LabelEncoder()
data['v1'] = l.fit_transform(data['v1'])
data['v1'].unique()

array([0, 1])

In [18]:
y = data['v1'].values
y = y.reshape(-1,1)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.4)

## 5. Model building - Adding layers, Compiling model, Fitting model and Saving model

### Add Layers (LSTM, Dense-(Hidden Layers), Output)

In [19]:
model = Sequential()

In [20]:
model.add(Dense(1565,activation = "relu"))
model.add(Dense(3000,activation = "relu"))
model.add(Dense(1,activation = "sigmoid"))
model.add(Flatten())

### Compile the Model

In [23]:
model.compile(optimizer = "adam",loss = "binary_crossentropy", metrics = ["accuracy"])

### Fit the Model

In [24]:
model.fit(x_train,y_train,epochs = 15)

Epoch 1/15
105/105 [==============================] - 9s 68ms/step - loss: 0.1361 - accuracy: 0.9626
Epoch 2/15
105/105 [==============================] - 7s 62ms/step - loss: 0.0175 - accuracy: 0.9946
Epoch 3/15
105/105 [==============================] - 7s 63ms/step - loss: 0.0039 - accuracy: 0.9991
Epoch 4/15
105/105 [==============================] - 7s 63ms/step - loss: 0.0022 - accuracy: 0.9997
Epoch 5/15
105/105 [==============================] - 7s 63ms/step - loss: 0.0020 - accuracy: 0.9997
Epoch 6/15
105/105 [==============================] - 7s 63ms/step - loss: 0.0019 - accuracy: 0.9997
Epoch 7/15
105/105 [==============================] - 8s 77ms/step - loss: 0.0018 - accuracy: 0.9997
Epoch 8/15
105/105 [==============================] - 7s 65ms/step - loss: 0.0018 - accuracy: 0.9997
Epoch 9/15
105/105 [==============================] - 9s 83ms/step - loss: 0.0017 - accuracy: 0.9997
Epoch 10/15
105/105 [==============================] - 7s 63ms/step - loss: 0.0017 - accura

## 6.Test the Model

In [25]:
ham = "im donee. come pick me up"
spam = "WINNER$$$$ SMS REPLY 'WIN'"


In [26]:
message = re.sub('[^a-zA-Z]',' ',spam)


In [27]:
message

'WINNER     SMS REPLY  WIN '

### Testing with spam message

In [28]:
message = message.split()
message = [ps.stem(word) for word in message if not word in set(stopwords.words('english')) ]
message = ' '.join(message)

In [29]:
message

'winner sm repli win'

In [30]:
message1 = cv.transform([message])
message1

<1x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [31]:
TruePredction = model.predict(message1.astype(float))

1/1 [==============================] - 0s 141ms/step


### Testing with normal message


In [32]:
msg = re.sub('[^a-zA-Z]',' ',ham)
msg

'im donee  come pick me up'

In [33]:
msg = msg.split()
msg = [ps.stem(word) for word in msg if not word in set(stopwords.words('english'))]
msg = ' '.join(msg)

In [34]:
msg

'im done come pick'

In [35]:
cv.transform([msg])

<1x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [36]:
FalsePredection = model.predict(cv.transform([msg]))

1/1 [==============================] - 0s 85ms/step


In [37]:
FalsePredection > 0.5

array([[False]])